In [1]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [2]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                 if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w, add_start_end=True):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    if add_start_end:
        w = '<start> ' + w + ' <end>'
    return w

In [3]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))


<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [4]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in line.split('\t')]
                for line in lines[:num_examples]]
    print(word_pairs)

    return zip(*word_pairs)


### Load recipes

In [5]:
import json

In [6]:
num_samples = 20000

In [7]:
with open('../dataset/layers/layer1.json', 'r') as f:
    dataset = json.load(f)[:num_samples]

In [8]:
with open("../dataset/layers/layer4.json", "r") as f:
    dataset3 = json.load(f)[:num_samples]

In [9]:
for record, ingredients in zip(dataset, dataset3):
    record['ingredients'] = ' '.join(ingredients)
    record['instructions'] = ' '.join([instruction['text'] for instruction in record['instructions']])

ingredients = [record['ingredients'].lower() for record in dataset]
instructions = [f"{record['instructions']}" for record in dataset]


In [10]:
len(ingredients), len(instructions)

(20000, 20000)

In [11]:
max_instructions_length = 100
length_mask = [len(instruction.split(" ")) <= max_instructions_length for instruction in instructions]
ingredients = [ingredient for i, ingredient in enumerate(ingredients) if length_mask[i]]
instructions = [instruction for i, instruction in enumerate(instructions) if length_mask[i]]


In [12]:
max_ingredients_length = 15
length_mask = [len(ingredient.split(" ")) <= max_ingredients_length for ingredient in ingredients]
ingredients = [ingredient for i, ingredient in enumerate(ingredients) if length_mask[i]]
instructions = [instruction for i, instruction in enumerate(instructions) if length_mask[i]]

min_ingredients_length = 5
length_mask = [len(ingredient.split(" ")) >= min_ingredients_length for ingredient in ingredients]
ingredients = [ingredient for i, ingredient in enumerate(ingredients) if length_mask[i]]
instructions = [instruction for i, instruction in enumerate(instructions) if length_mask[i]]

In [13]:
len(ingredients), len(instructions)

(8935, 8935)

In [14]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

In [15]:
def load_dataset(instructions, ingredients):
    targ_lang = [preprocess_sentence(instr, True) for instr in instructions]
    inp_lang = [preprocess_sentence(ingr, False) for ingr in ingredients]
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [16]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(instructions, ingredients)
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [17]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=42)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))


7148 7148 1787 1787


In [18]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(f'{t} ----> {lang.index_word[t]}')

In [19]:
print("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print()
print("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])


Input Language; index to word mapping
65 ----> sageleaf
24 ----> carrot
8 ----> water
11 ----> lemon
1011 ----> polenta
1 ----> salt

Target Language; index to word mapping
10 ----> <start>
1228 ----> quarter
104 ----> lemon
3 ----> and
58 ----> remove
278 ----> seeds
1 ----> .
514 ----> finely
263 ----> chop
484 ----> thyme
320 ----> leaves
3 ----> and
37 ----> place
8 ----> with
104 ----> lemon
5 ----> in
40 ----> medium
98 ----> saucepan
1 ----> .
12 ----> add
247 ----> carrots
3 ----> and
42 ----> water
2 ----> ,
82 ----> bring
6 ----> to
7 ----> a
61 ----> boil
3 ----> and
23 ----> cook
9 ----> until
736 ----> reduced
6 ----> to
202 ----> cups
1 ----> .
43 ----> pour
5 ----> in
1405 ----> polenta
5 ----> in
7 ----> a
416 ----> thin
957 ----> stream
2 ----> ,
750 ----> whisking
284 ----> constantly
2 ----> ,
3 ----> and
59 ----> then
23 ----> cook
2 ----> ,
7020 ----> switching
6 ----> to
7 ----> a
763 ----> wooden
119 ----> spoon
118 ----> as
1405 ----> polenta
630 ----> thickens


In [20]:
input_tensor_train.shape

(7148, 18)

In [21]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 128
units = 256
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


In [22]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 18]), TensorShape([64, 136]))

In [23]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))


In [24]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units)', sample_output.shape)
print('Encoder Hidden state shape: (batch size, units)', sample_hidden.shape)


Encoder output shape: (batch size, sequence length, units) (64, 18, 256)
Encoder Hidden state shape: (batch size, units) (64, 256)


In [25]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [26]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units)", attention_result.shape)
print("Attention weights shape: (batch_size, sequence_length, 1)", attention_weights.shape)


Attention result shape: (batch size, units) (64, 256)
Attention weights shape: (batch_size, sequence_length, 1) (64, 18, 1)


In [27]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform'
                                       )
        self.fc = tf.keras.layers.Dense(vocab_size, activation='softmax')

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [28]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)



Decoder output shape: (batch_size, vocab size) (64, 7058)


In [29]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


In [30]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


In [31]:
## @tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [32]:
EPOCHS = 100

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix=checkpoint_prefix)

    print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')


Epoch 1 Batch 0 Loss 4.2691
Epoch 1 Batch 100 Loss 4.4131
Epoch 1 Loss 4.5067
Time taken for 1 epoch 148.31 sec

Epoch 2 Batch 0 Loss 4.5393
Epoch 2 Batch 100 Loss 4.3236
Epoch 2 Loss 4.4910
Time taken for 1 epoch 148.78 sec

Epoch 3 Batch 0 Loss 4.5666
Epoch 3 Batch 100 Loss 4.1009
Epoch 3 Loss 4.4888
Time taken for 1 epoch 148.21 sec

Epoch 4 Batch 0 Loss 4.8825
Epoch 4 Batch 100 Loss 4.5682
Epoch 4 Loss 4.4899
Time taken for 1 epoch 148.79 sec

Epoch 5 Batch 0 Loss 4.1589
Epoch 5 Batch 100 Loss 4.4579
Epoch 5 Loss 4.4892
Time taken for 1 epoch 148.95 sec

Epoch 6 Batch 0 Loss 4.7732
Epoch 6 Batch 100 Loss 4.3436
Epoch 6 Loss 4.4914
Time taken for 1 epoch 149.92 sec

Epoch 7 Batch 0 Loss 4.4990
Epoch 7 Batch 100 Loss 4.3723
Epoch 7 Loss 4.4894
Time taken for 1 epoch 149.66 sec

Epoch 8 Batch 0 Loss 4.7909
Epoch 8 Batch 100 Loss 4.6130
Epoch 8 Loss 4.4873
Time taken for 1 epoch 148.04 sec

Epoch 9 Batch 0 Loss 5.0133
Epoch 9 Batch 100 Loss 4.5194
Epoch 9 Loss 4.4910
Time taken for 1 e

Epoch 72 Batch 0 Loss 4.4036
Epoch 72 Batch 100 Loss 4.5947
Epoch 72 Loss 4.4898
Time taken for 1 epoch 148.93 sec

Epoch 73 Batch 0 Loss 4.4734
Epoch 73 Batch 100 Loss 4.1766
Epoch 73 Loss 4.4881
Time taken for 1 epoch 148.80 sec

Epoch 74 Batch 0 Loss 4.4676
Epoch 74 Batch 100 Loss 4.7054
Epoch 74 Loss 4.4866
Time taken for 1 epoch 148.57 sec

Epoch 75 Batch 0 Loss 4.2658
Epoch 75 Batch 100 Loss 4.4012
Epoch 75 Loss 4.4887
Time taken for 1 epoch 148.99 sec

Epoch 76 Batch 0 Loss 4.8480
Epoch 76 Batch 100 Loss 4.2038
Epoch 76 Loss 4.4878
Time taken for 1 epoch 148.74 sec

Epoch 77 Batch 0 Loss 4.2871
Epoch 77 Batch 100 Loss 4.2921
Epoch 77 Loss 4.4915
Time taken for 1 epoch 148.68 sec

Epoch 78 Batch 0 Loss 4.3695
Epoch 78 Batch 100 Loss 4.5401
Epoch 78 Loss 4.4911
Time taken for 1 epoch 149.01 sec

Epoch 79 Batch 0 Loss 4.3524
Epoch 79 Batch 100 Loss 4.5788
Epoch 79 Loss 4.4885
Time taken for 1 epoch 149.40 sec

Epoch 80 Batch 0 Loss 4.9015
Epoch 80 Batch 100 Loss 4.3413
Epoch 80 Los

In [46]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [34]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence, False)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = sample(predictions[0])

#         predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot




In [35]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


In [36]:
def translate(sentence, verbose=False):
    result, sentence, attention_plot = evaluate(sentence)

    if verbose:
        print('Input:', sentence)
        print('Predicted translation:', result)

        attention_plot = attention_plot[:len(result.split(' ')),
                                      :len(sentence.split(' '))]
        plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    return result


In [49]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt-50'

In [37]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))


In [38]:
instructions[0]

'Cook macaroni according to package directions; drain well. Cold. Combine macaroni, cheese cubes, celery, green pepper and pimento. Blend together mayonnaise or possibly salad dressing, vinegar, salt and dill weed; add in to macaroni mix. Toss lightly. Cover and refrigeratewell. Serve salad in lettuce lined bowl if you like. Makes 6 servings.'

In [39]:
translate(ingredients[102])

[1.20653614e-11 9.99999830e-01 2.73159635e-08 ... 1.48696360e-11
 1.20092492e-11 1.23511076e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01

[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01 2.72718690e-08 ... 1.48286531e-11
 1.19774526e-11 1.23187333e-11]
[1.20325448e-11 9.99999830e-01

'. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . '

### Evaluating the model

In [40]:
num_examples = 1000
input_tensors = input_tensor_val[:num_examples]
target_tensors = target_tensor_val[:num_examples]

In [41]:
def convert_to_string(tensors, lang):
    texts = []
    for tensor in tensors:
        text = ''
        for t in tensor:
            if t != 0:
                text += lang.index_word[t] + ' '
        texts.append(text[:-1])
    return texts

In [42]:
input_strings = convert_to_string(input_tensors, inp_lang)
target_strings = convert_to_string(target_tensors, targ_lang)

In [47]:
from tqdm import tqdm
generated_texts = []
for ingrs in tqdm(input_strings):
    generated_texts.append(translate(ingrs))

100%|██████████| 1000/1000 [10:47<00:00,  1.55it/s]


In [48]:
generated_texts[:5]

['. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ',
 '. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ',
 '. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ',
 '. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

#### Using rouge

In [ ]:
import string
def preprocess_text_for_rouge(samples):
    samples = [text.translate(str.maketrans('', '', string.punctuation)) for text in samples]
    samples = [text.lower() for text in samples]
    samples = [' '.join(text.split()) for text in samples]
#     samples = [text.split() for text in samples]
    return samples

In [ ]:
for i in range(len(generated_texts)):
    generated_texts[i] = 'put sour pancake to boil for 15 to 20 minutes'

In [ ]:
generated_text_rouge = preprocess_text_for_rouge(generated_texts)
target_text_rouge = preprocess_text_for_rouge(target_strings)

In [ ]:
generated_text_rouge[0]

In [ ]:
import rouge

rouge = rouge.Rouge()
rouge.get_scores(generated_text_rouge, target_text_rouge, avg=True)

#### Using bleu

In [ ]:
import string
def preprocess_generated_text_for_bleu(samples):
    samples = [text.translate(str.maketrans('', '', string.punctuation)) for text in samples]
    samples = [text.lower() for text in samples]
    samples = [' '.join(text.split()) for text in samples]
    samples = [text.split() for text in samples]
    return samples

import string
def preprocess_reference_text_for_bleu(samples):
    samples = [text.translate(str.maketrans('', '', string.punctuation)) for text in samples]
    samples = [text.lower() for text in samples]
    samples = [' '.join(text.split()) for text in samples]
    samples = [[text.split()] for text in samples]
    return samples

In [ ]:
generated_text_bleu = preprocess_generated_text_for_bleu(generated_texts)
target_text_bleu = preprocess_reference_text_for_bleu(target_strings)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

score_1_gram = corpus_bleu(target_text_bleu, generated_text_bleu, weights=(1, 0, 0, 0))
score_2_gram = corpus_bleu(target_text_bleu, generated_text_bleu, weights=(0, 1, 0, 0))

In [ ]:
score_1_gram, score_2_gram